# Visualization

In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import os
import json

In [ ]:
def visual_per_class(data, savepath):
    colors = [(1, 0, 0), (1, 1, 0.5), (0, 1, 0)]  # red, orange, green
    cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)

    # map accuracy 0~1 to red, orange, green gradient
    acc_cols = [col for col in data.columns if col.startswith("Top-") and col.endswith("Acc")]
    vis = data.style.background_gradient(subset=acc_cols, cmap=cmap, vmin=0, vmax=1)
    vis = vis.set_properties(**{'color': 'black'}, subset=acc_cols)

    # save
    vis.to_excel(savepath, index=False)
    return vis

In [ ]:
def rgb_to_hex(color):
    rgb_int = (color * 255).astype(int)
    return '#{:02X}{:02X}{:02X}'.format(*rgb_int)

def make_green_colormap(prob):
    white = np.array([1, 1, 1])
    green = np.array([0, 1, 0])
    alpha = np.clip(prob, 0, 1)
    color = (1 - alpha) * white + alpha * green
    return f"background-color: {rgb_to_hex(color)}; color: black"

def make_red_colormap(prob):
    white = np.array([1, 1, 1])
    red = np.array([1, 0, 0])
    alpha = np.clip(prob, 0, 1)
    color = (1 - alpha) * white + alpha * red
    return f"background-color: {rgb_to_hex(color)}; color: black"

def visual_per_image(data, savepath):
    label_cols = [f"Top{i}_Label" for i in range(1, 6)]
    prob_cols = [f"Top{i}_Prob" for i in range(1, 6)]

    label_df = data[["Image", "True Label"] + label_cols].copy()
    prob_df = data[["Image", "True Label"] + prob_cols].copy()

    df_vis = label_df.copy()
    for col in prob_cols:
        df_vis[col] = prob_df[col]

    # compare labels with truth to determine colors
    def style_func(row):
        styles = []
        true_label = row["True Label"]
        for i in range(1, 6):
            pred_label = row[f"Top{i}_Label"]
            prob = row[f"Top{i}_Prob"]
            if pred_label == true_label:
                styles.append(make_green_colormap(prob))
            else:
                styles.append(make_red_colormap(prob))
        return pd.Series(styles, index=label_cols)

    styled = df_vis.style.apply(style_func, axis=1)
    final_cols = ["Image", "True Label"] + label_cols
    styled = styled.hide(axis="columns", subset=[col for col in df_vis.columns if col not in final_cols])

    # save
    styled.to_excel(savepath, index=False)
    return styled

### Zeroshot Results

In [96]:
zero_perclass = pd.read_csv('rs/zeroshotRS/per_class_top5_accuracy.csv')
zeroVis = visual_per_class(zero_perclass, "rs/zeroshotRS/per_class_top5_accuracy.xlsx")
zeroVis

,Class,Top-1 Acc,Top-2 Acc,Top-3 Acc,Top-4 Acc,Top-5 Acc
0,animal crossing,0.300000,0.500000,0.600000,0.700000,0.800000
1,bumpy road,0.000000,0.000000,0.400000,0.700000,0.900000
2,cross road,0.400000,0.700000,0.900000,1.000000,1.000000
3,cycle crossing,1.000000,1.000000,1.000000,1.000000,1.000000
4,dip,0.000000,0.000000,0.000000,0.000000,0.000000
5,gas station,1.000000,1.000000,1.000000,1.000000,1.000000
6,give way,0.400000,0.550000,0.550000,0.700000,0.850000
7,height limit,0.000000,0.600000,1.000000,1.000000,1.000000
8,hospital,1.000000,1.000000,1.000000,1.000000,1.000000
9,left turn,0.000000,0.850000,1.000000,1.000000,1.000000


In [97]:
zero_perimage = pd.read_csv('rs/zeroshotRS/per_image_top5.csv')
zeroVis = visual_per_image(zero_perimage, "rs/zeroshotRS/per_image_top5.xlsx")
# zeroVis

### Finetune Results

In [98]:
fine_perclass = pd.read_csv('rs/finetuneRS/per_class_top5_accuracy.csv')
fineVis = visual_per_class(fine_perclass, "rs/finetuneRS/per_class_top5_accuracy.xlsx")
fineVis

,Class,Top-1 Acc,Top-2 Acc,Top-3 Acc,Top-4 Acc,Top-5 Acc
0,animal crossing,1.000000,1.000000,1.000000,1.000000,1.000000
1,bumpy road,1.000000,1.000000,1.000000,1.000000,1.000000
2,cross road,1.000000,1.000000,1.000000,1.000000,1.000000
3,cycle crossing,1.000000,1.000000,1.000000,1.000000,1.000000
4,dip,1.000000,1.000000,1.000000,1.000000,1.000000
5,gas station,1.000000,1.000000,1.000000,1.000000,1.000000
6,give way,1.000000,1.000000,1.000000,1.000000,1.000000
7,height limit,1.000000,1.000000,1.000000,1.000000,1.000000
8,hospital,1.000000,1.000000,1.000000,1.000000,1.000000
9,left turn,1.000000,1.000000,1.000000,1.000000,1.000000


In [99]:
fine_perimage = pd.read_csv('rs/finetuneRS/per_image_top5.csv')
fineVis = visual_per_image(fine_perimage, "rs/finetuneRS/per_image_top5.xlsx")
# fineVis